<a href="https://colab.research.google.com/github/YOONSEOUNGHWAN/AI_TEST/blob/main/KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SETTING

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-m1_nrjhr
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-m1_nrjhr
     |████████████████████████████████| 132 kB 5.0 MB/s 
     |████████████████████████████████| 344 kB 45.4 MB/s 
     |████████████████████████████████| 49.1 MB 149 kB/s 
     |████████████████████████████████| 4.5 MB 16.1 MB/s 
     |████████████████████████████████| 1.2 MB 52.8 MB/s 
     |████████████████████████████████| 4.4 MB 27.4 MB/s 
     |████████████████████████████████| 596 kB 58.1 MB/s 
     |████████████████████████████████| 6.6 MB 56.9 MB/s 
     |████████████████████████████████| 101 kB 10.8 MB/s 
     |████████████████████████████████| 9.0 MB 58.3 MB/s 
     |████████████████████████████████| 79 kB 5.7 MB/s 
     |████████████████████████████████| 127 kB 69.8 MB/

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:

## GPU
device = torch.device("cuda:0")

In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
chatbot_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/DATA_REAL_FINAL.xlsx')
# chatbot_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/가공DATA.xlsx')



In [ ]:
chatbot_data

,Sentence,Emotion
0,B형 간염 항체가 없다고하네. 예방접종을 해야되나?,Embarressed
1,힘없이 늘어져 있는 것이 싫어서 힘이 넘치는 척하면서 바깥을 돌아다녀. 현실에 너무...,Angry
2,힘없이 늘어져 있는 것이 싫어서 힘이 넘치는 척하면서 바깥을 돌아다녀. 현실에 너무...,Angry
3,힘들었지만 열심히 해준 팀원들 덕분에 프로젝트가 성공적이어서 기뻤어.,Happy
4,힘들었지만 열심히 해준 팀원들 덕분에 프로젝트가 성공적이어서 기뻤어.,Happy
...,...,...
140216,경로당 사람들이 내가 자식한테 버림 받았다는 사실을 알게 돼서 너무 창피하고 슬퍼.,Sad
140217,건강 검진 결과를 보고 이제껏 내 건강에 자부심을 느끼고 살았었는데 너무 놀랐어.,Embarressed
140218,\n우리 세대를 보면 시부모님 공양하고 애들 뒷바라지 하느라 노후준비하기가 정말 어...,Anxiety
140219,\n옆집 어르신들이 자식들한테 모은 돈 다 퍼주시고 힘들게 사시는 모습 보면 너무 ...,Sad


In [ ]:
chatbot_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140221 entries, 0 to 140220
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Sentence  140221 non-null  object
 1   Emotion   140221 non-null  object
dtypes: object(2)
memory usage: 2.1+ MB


In [ ]:
chatbot_data.loc[(chatbot_data['Emotion'] == 'Normal'), 'Emotion'] = 0    #중립 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == 'Angry'), 'Emotion'] = 1     #분노 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == 'Sad'), 'Emotion'] = 2       #슬픔 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == 'Happy'), 'Emotion'] = 3      #행복 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == 'Embarressed'), 'Emotion'] = 4   #당황 => 4
chatbot_data.loc[(chatbot_data['Emotion'] == 'Anxiety'), 'Emotion'] = 5      #불안 => 5


In [ ]:

chatbot_data.isnull().sum()

Sentence    0
Emotion     0
dtype: int64

In [ ]:
chatbot_data = chatbot_data.dropna(how='any')

In [ ]:
data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion']):
    data = []
    data.append(q)
    data.append(str(label))
    # print(data)
    data_list.append(data)

len(data_list)

140221

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, random_state=11)
print(len(dataset_train))
print(len(dataset_test))

105165
35056


In [ ]:
# dataset_train

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
dataset_train[:20]

[['나는 항상 어떤 일에든 최선의 노력을 했어. 내게 없는 것을 탓하지 않았어.', '3'],
 ['나는 정말 지금 너무 억울에 갑자기 암에 걸렸어.', '1'],
 ['지금 재직 중인 직장을 계속 다녀야 하나 싶네.', '2'],
 ['와이프가 옆에 있어 너무 든든하고 고마워.', '3'],
 ['우리 큰 딸 때문에 너무 속상한 일이 있어.', '2'],
 ['나 진짜 내 친구들한테 실망했어. 나 빼고 어디 놀러 갔어.', '2'],
 ['내가 무슨 직업을 가져야 할지 모르겠어서 초조해.', '5'],
 ['어쩜 이럴 수 있지? 남편이 나를 두고 바람을 피웠어.', '1'],
 ['돈이 없어서 당혹스러워.', '4'],
 ['친하게 지내던 친구들이 나를 빼고 다니기 시작하더니 갑자기 나를 따돌려. 외롭고 힘들어.', '2'],
 ['지금 투자를 많이 했는데 수익이 안 나고 있.어', '5'],
 ['직장 상사가 자꾸만 내 언급을 다른 사람에게 한다는 얘기가 들려서 성가셔.', '1'],
 ['학교에서 가해자들과 나를 다른반으로 해주었어', '3'],
 ['내가 갑자기 따돌림을 당하게 되었어. 갑작스러운 일이라 너무 놀랍고 혼란스러워.', '4'],
 ['내일이 최종 면접일인데 내가 잘 해낼 수 있을지 걱정스러워. 너무 떨려서 제대로 말을 못 하고 오면 어떡하지?', '5'],
 ['평소 눈 건강에는 자신이 있었는데 노안이 왔다는 거야. 정말 당황했어.', '2'],
 ['골수 기증만 받으면 살릴 수 있는데 무섭고 아프다고 기증을 거부하는 친구의 모습은 구역질 날 정도 실망스럽다.', '1'],
 ['내 동생만 귀염받아서 질투가 나.', '1'],
 ['자식들 다 결혼시키고 귀농하려고 하는데 막내가 장가를 안 간대. 내 노후 꿈을 아들 녀석이 방해하고 있어.', '2'],
 ['평소에 명상하는데 좀 더 깊게 해봐야겠어.', '0']]

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
data_train[0]

(array([   2, 1375, 4996, 3224, 3803, 6896, 5928, 4519, 6559, 7095, 1482,
        5017, 6855,  517,   54, 1434, 5400, 3272,  911, 4717, 7819, 3156,
        6855,  517,   54,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(26, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/1644 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9153895378112793 train acc 0.125
epoch 1 batch id 201 loss 1.614302635192871 train acc 0.29578669154228854
epoch 1 batch id 401 loss 1.2093751430511475 train acc 0.3981842269326683
epoch 1 batch id 601 loss 1.1564146280288696 train acc 0.47103785357737105
epoch 1 batch id 801 loss 0.8531584739685059 train acc 0.5117041198501873
epoch 1 batch id 1001 loss 0.9805007576942444 train acc 0.5399288211788211
epoch 1 batch id 1201 loss 0.879082441329956 train acc 0.5576342631140716
epoch 1 batch id 1401 loss 0.922467052936554 train acc 0.5714333511777302
epoch 1 batch id 1601 loss 1.2030088901519775 train acc 0.5816286695815116
epoch 1 train acc 0.583565091006925


  0%|          | 0/548 [00:00<?, ?it/s]

epoch 1 test acc 0.6671894008515815


...

epoch 9 test acc 0.8027106143552312


  0%|          | 0/1644 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.21449680626392365 train acc 0.9375
epoch 10 batch id 201 loss 0.16953906416893005 train acc 0.9636194029850746
epoch 10 batch id 401 loss 0.21340686082839966 train acc 0.9650483167082294
epoch 10 batch id 601 loss 0.12165827304124832 train acc 0.9634203410981698
epoch 10 batch id 801 loss 0.1032191663980484 train acc 0.9636001872659176
epoch 10 batch id 1001 loss 0.1711057424545288 train acc 0.9641296203796204
epoch 10 batch id 1201 loss 0.06029660254716873 train acc 0.9641574729392173
epoch 10 batch id 1401 loss 0.16993141174316406 train acc 0.9646346359743041
epoch 10 batch id 1601 loss 0.16188739240169525 train acc 0.9643972517176764
epoch 10 train acc 0.9645681265206812


  0%|          | 0/548 [00:00<?, ?it/s]

epoch 10 test acc 0.8040507147201947


In [ ]:
## 학습 모델 저장
PATH = '/content/drive/MyDrive/Colab Notebooks/' 
torch.save(model, PATH + 'KoBERT_감정분류.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능